In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
path = os.getcwd() + "/atribuna/aTribuna_completo"
content_file_array = []
classifier_array = []

In [10]:
def read_text(path, file_Name):
    with open(path, encoding="latin-1") as file:
        content_file = file.read()
        Build_DataFrame(content_file, file_Name)

In [11]:
def Build_DataFrame(content_file, file_Name):
    classifier =  GetClassifier(file_Name) 
    if(len(classifier) >= 3):        
        classifier_array.append(classifier)
        content_file_array.append(content_file)  

In [12]:
def GetClassifier(file_Name):
    name_split =  file_Name.split(".")
    first_name = name_split[0];
    return first_name[(len(first_name) - 3) : len(first_name)]

In [13]:
for file_Name in os.listdir(path):    
    if file_Name.endswith(".txt"):
        file_path = f"{path}\{file_Name}"
      
        # call read text file function
        read_text(file_path, file_Name)

In [14]:
textDataDrame =  pd.DataFrame({"Text":content_file_array, "Class":classifier_array})


In [15]:
textDataDrame.head(10)

,Text,Class
0,Testes nas famílias\n<b> A pedido de A Tribuna...,at2
1,"Deu borboleta na moda\n<b> A borboleta, que fo...",at2
2,Roupas de malha vão à festa\n<b> Se antes as p...,at2
3,Chega de dor-de-cotovelo\n<b> Com muito humor ...,at2
4,Tom Cruise volta com tudo\n<b> Ação sem fim e ...,at2
5,Temporada de arraiás\n<b> Sejam as tradicionai...,at2
6,"Bregas ontem, chiques hoje\n<b> Ídolos no pass...",at2
7,Picape boa de briga\n<b> Testada por Sobre Rod...,sro
8,Rita Cadilac vai se aposentar\n<b> A ex-chacre...,at2
9,"Cada balada, um visual\n<b> A turma das raves ...",at2
